### IMPORT STUFF HERE

In [ ]:
pip install geopy

In [784]:
%%time
# Importing all the necessary libraries for this project.

import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import datetime
import datetime as dt
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
import statsmodels.api as sm
from sklearn import metrics
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
import re
from selenium.webdriver.common.keys import Keys
from uuid import uuid4 as uuid
from datetime import datetime 
from pygeocoder import Geocoder
from time import sleep


CPU times: user 59 µs, sys: 101 µs, total: 160 µs
Wall time: 170 µs


In [887]:
wnv_data = pd.read_csv("train.csv", error_bad_lines=False, low_memory=False)
test = pd.read_csv("test.csv", error_bad_lines=False, low_memory=False)


wnv_spray = pd.read_csv("spray.csv", error_bad_lines=False, low_memory=False)

wnv_weather = pd.read_csv("weather.csv", error_bad_lines=False, low_memory=False)

census_info = pd.read_csv("census_info.csv", error_bad_lines=False, low_memory=False)

census_age = pd.read_csv("census_age.csv", error_bad_lines=False, low_memory=False)

cols = ['2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016']
robin= pd.DataFrame([[7502,6923,6543,6123,6915,7003,7907,7343,7385,6814,6341,6538]],columns=cols)

### FIX CENSUS_INFO columns & dtypes

In [888]:
census_info = census_info.drop(["GEO.id", "GEO.display-label", "Unnamed: 0"], axis = 1)
census_info["zip_code"] = census_info["zip_code"].astype(str)

census_age = census_age.drop(["GEO.id", "GEO.display-label", "Unnamed: 0"], axis = 1)
census_age["zip_code"] = census_age["zip_code"].astype(str)

In [891]:
test["Date"].value_counts()

2012-07-09    1293
2012-07-27    1282
2012-08-03    1282
2012-07-19    1260
2010-07-13    1257
2012-07-13    1256
2014-08-14    1254
2014-08-21    1253
2008-09-09    1248
2008-09-02    1247
2008-08-19    1247
2014-08-07    1247
2012-07-20    1245
2014-07-31    1245
2010-07-19    1241
2014-06-19    1236
2014-08-28    1234
2008-07-24    1234
2008-08-13    1233
2014-07-10    1233
2008-08-26    1232
2012-08-10    1231
2012-06-29    1231
2010-07-26    1229
2014-07-03    1229
2012-08-17    1229
2014-09-05    1229
2010-06-28    1227
2014-06-12    1225
2010-07-30    1225
              ... 
2010-09-24    1214
2012-09-20    1214
2012-06-15    1214
2008-08-12    1214
2008-08-04    1213
2012-06-08    1213
2008-09-03    1213
2012-08-09    1213
2010-06-02    1212
2010-06-11    1211
2010-07-29    1211
2008-07-01    1211
2008-09-19    1211
2008-07-28    1210
2010-08-26    1210
2010-09-10    1210
2008-07-04    1209
2010-08-05    1209
2008-07-11    1209
2012-08-30    1209
2010-06-18    1209
2010-08-19  

In [892]:
def zip_code(x):
    if "60" in x.split()[-2]:
        zipcode = x.split()[-2].replace(",", "")
    elif "South Doty Avenue, Chicago, IL, USA" in x:
        zipcode = "60633"
    elif "South Stony Island Avenue, Chicago, IL, USA" in x:
        zipcode = "60617"
    elif "South Torrence Avenue, Chicago, IL, USA" in x:
        zipcode = "60617"
    elif "South Cottage Grove Avenue, Chicago, IL, USA" in x:
        zipcode = "60619"
    elif "South Vincennes Avenue, Chicago, IL, USA" in x:
        zipcode = "60620"
    elif "East 91st Place, Chicago, IL, USA" in x:
        zipcode = "60619"
    elif "West Garfield Boulevard, Chicago, IL, USA" in x:
        zipcode = "60636"
    elif "6400 West Strong Street, Chicago, IL, USA" in x:
        zipcode = "60630"
    elif "East 91st Street, Chicago, IL, USA" in x:
        zipcode = "60617"
    elif "West 37th Place, Chicago, IL, USA" in x:
        zipcode = "60609"
    elif "South Wallace Street, Chicago, IL, USA" in x:
        zipcode = "60620"
    elif x == "Chicago, IL, USA":
        zipcode = "60613"
    else:
        zipcode = "00000"
    
    return(zipcode)

def fix_date(x):
    fix_date = datetime.strptime(x, "%Y-%m-%d")
    return(fix_date)

In [915]:
## FIX DATE, ZIPCODE HERE

In [893]:
wnv_data["Date"] = wnv_data["Date"].apply(fix_date)

In [894]:
wnv_data["ZipCode"] = wnv_data["Address"].apply(zip_code)

In [895]:
wnv_data[wnv_data["ZipCode"]=="00000"]["AddressNumberAndStreet"].value_counts()

Series([], Name: AddressNumberAndStreet, dtype: int64)

In [896]:
wnv_data.dtypes

Date                      datetime64[ns]
Address                           object
Species                           object
Block                              int64
Street                            object
Trap                              object
AddressNumberAndStreet            object
Latitude                         float64
Longitude                        float64
AddressAccuracy                    int64
NumMosquitos                       int64
WnvPresent                         int64
ZipCode                           object
dtype: object

In [897]:
census_age = census_age.replace("-", 0)

census_age.zip_code = census_age.zip_code.astype(object)
census_age.total_population = census_age.total_population.astype(int)

In [898]:
census_age.head()

,zip_code,total_population,percent_age_under_5,percent_age_5-9,percent_age_10-14,percent_age_15-19,percent_age_20-24,percent_age_25-34,percent_age_35-44,percent_age_45-54,percent_age_55-59,percent_age_60-64,percent_age_65-74,percent_age_75-84,percent_age85_and_over,percent_age_65_and_over,median_age
0,60002,24250,5.7,7.2,6.4,7.9,4.7,9.9,14.2,17.3,8.1,6.5,8.2,2.8,1,12,41.2
1,60004,49957,5.5,6.5,6,6.3,4.8,11.8,13.1,16.1,6.4,6.5,8.9,5.6,2.6,17.1,41.8
2,60005,30931,7,5.1,7.3,6.1,4.4,13.5,14.9,14.8,4.9,5.9,7.5,5.2,3.4,16,39.8
3,60007,33973,4.6,4.9,6.5,6.3,5.7,11.3,12.9,18.4,7.6,6.9,7.7,5.4,1.8,14.9,43.5
4,60008,22302,8.1,6.2,5.2,6,5.8,16.6,13,14.9,6.4,4.9,6.2,4.5,2,12.8,36.2


#### MERGE WNV + CENSUS = WNV_POP. THEN MERGE WNV_POP + CENSUS_AGE = WNV_POP_AGE. THEN MERGE MERGE WNV_POP_AGE + SPRAY = WNV_SPRAY

In [910]:
wnv_pop = pd.merge(wnv_data, census_info, how = "left", left_on=["ZipCode"], right_on=["zip_code"])
wnv_pop = wnv_pop.drop("zip_code", axis = 1)

In [932]:
wnv_pop_age = pd.merge(wnv_pop, census_age, how = "left", left_on=["ZipCode"], right_on=["zip_code"])
wnv_pop_age = wnv_pop_age.drop("zip_code", axis = 1)

In [954]:
wnv_pop_age.to_csv('WNV_Pop_Age.csv', mode='a', header=True)